In [8]:
import gc
import os
import glob
import os.path

import dask
import dask.array as da
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster, progress

gc.collect()

os.chdir(
    "/mnt/cloud/wwu1/ec_bronze/_nogroup/ae78a1ca-a0e8-4e4e-8992-69c34947db65/UseCase_AIRCON"
)


# relative humidity
def rh(t, d):
    t = t - 273.15 # Kelvin to °C
    d = d - 273.15
    es = 6.1078 * da.exp((17.1 * t) / (235 + t))
    e = 6.1078 * da.exp((17.1 * d) / (235 + d))
    return (e / es) * 100
    
#scale_factor_rh = 100 / (2**16 - 1)  # applies to all datasets

years = range(2015, 2024)

In [18]:
t = 250
d = 246
print(rh(t, d))
rh2(t, d)

72.57026994762613


69.41659597418506

In [19]:
client = Client()
client

/home/j/jheisig/.local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39483 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39483/status,
Dashboard: http://127.0.0.1:39483/status,Workers: 4
Total threads: 16,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40213,Workers: 4
Dashboard: http://127.0.0.1:39483/status,Total threads: 16
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:35225,Total threads: 4
Dashboard: http://127.0.0.1:44725/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:39899,


In [20]:
def rel_hum(y):
    
    out = f"supplementary/era5_download/ERA5_2_metre_rel_humidity_{y}.nc"
    if not os.path.isfile(out) or os.path.getsize(out) < 50000:
        var_h = xr.open_mfdataset(
            [f"supplementary/era5_download/ERA5_2m_temperature_hrly_{y}.nc",
             f"supplementary/era5_download/ERA5_2m_dewpoint_temperature_hrly_{y}.nc"],
            chunks="auto",
            combine="by_coords",
            parallel=True,
        )
        
        if 'expver' in var_h.dims:
            var_h = var_h.reduce(np.nansum, 'expver')
            print("\nFound experimental version of temperature data (recent dates). Reducing dimension.")

        var_h["rh"] = rh(var_h.t2m, var_h.d2m)
        res = var_h.rh.to_netcdf(out, compute=False)   
        gc.collect()
        return res
        del res

In [21]:
%%time

lazy_results = []

for y in years:
    rh_d = rel_hum(y)
    lazy_results.append(rh_d)
    print(y, sep=" ", end=" ", flush=True)
    

2015 2016 2017 2018 2019 2020 2021 2022 
Found experimental version of temperature data (recent dates). Reducing dimension.
2023 CPU times: user 3.79 s, sys: 546 ms, total: 4.34 s
Wall time: 21.6 s


In [22]:
lazy_results = [x for x in lazy_results if x is not None]
len(lazy_results)

9

In [23]:
daily_res = dask.persist(*lazy_results)
progress(daily_res)

VBox()